# Problemas de alocação

Os problemas de alocação modelam uma relação entre dois tipos de entidades: *compromissos* e *recursos*. Adicionalmente, as restrições sobre a alocação de recursos a compromissos classificam-se em dois tipos: *obrigações* e *limitações*.

Nesta aula vamos considerar um problema de alocação. Pretende-se que faça a modelação do problema em Programação Inteira e que use o Z3 para os resolver.

Em geral, num solver para programação inteira todas as restrições devem ser equações lineares da forma $a_1 \cdot x_1 + \ldots a_n \cdot x_n \sim b$ onde $a_1, \ldots, a_n, b$ são constantes inteiras, $x_1, \ldots, x_n$ são variáveis inteiras e $\sim$ é um dos operadores de comparação $<,\leq,=,\geq,>$. Nesta (e na próxima) aula vamos tentar usar apenas este tipo de restrições para que as soluções desenvolvidas possam ser implementadas também noutros solveres para programação inteira, como por exemplo o SCIP.

In [83]:
from z3 import *

## Horário de um centro de estudos

Um centro de estudos possui $S$ salas de aula, $P$ professores, e está aberto $H$ horas por dia, durante $D$ dias na semana.

O centro de estudos funciona com as seguintes regras:
- Cada professor não pode dar mais do que $C$ horas por dia.
- Todos os professores do centro devem ter uma carga horária semanal parecida (diferença máxima de 2 horas).
- Não é permitido mais do que um professor por sala.
- Alocações contíguas do mesmo professor têm de ser na mesma sala.

Pretende-se estabelecer um horário para o centro de estudos que permita ter um número mínimo de aulas $N$ por semana.

### Análise do problema

Este é um problema de alocação. Pretende-se alocar professores a salas de aula, ao longo da semana, sendo o tempo de ocupação das salas de uma hora. 

Existem $P$ professores, que podemos identificar por um índice $p \in [0..P\!-\!1]$, e podemos identificar cada sala disponível num dado dia, a uma dada hora, por um triplo $(s,d,h) \in [0..S\!-\!1]\times[0..D\!-\!1]\times[0..H\!-\!1]$.

Vamos usar uma família $x_{p,s,d,h}$ de variáveis binárias (i.e., que assumem valores inteiros $\{0,1\}$), com a seguinte semântica

$$x_{p,s,d,h} == 1  \quad \mbox{se e só se} \quad \mbox{o professor $p$ for alocado à sala $s$, no dia $d$, à hora $h$.}$$

Estas $P\times S\times D\times H$ variáveis são convenientemente representadas numa matriz $X$ instanciável com valores $\{0,1\}^{P\times S\times D\times H}$, a que se costuma chamar *matriz de alocação*.

Destaca-se ainda o seguinte:

**Limitações** (que impõem limites máximos à alocação)

1. A carga horária diária máxima de cada professor é $C$.
2. Cada sala tem alocado, no máximo, um professor.
3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.
4. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.


**Obrigações** (que impõem limites mínimos à alocação)

5. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).
6. O número mínimo de aulas por semana é $N$.

### Implementação

Começamos por inicializar o solver *horario* e definir os valores para as constantes $S$, $P$, $H$, $D$, $C$ e $N$.

In [84]:
horario = Solver()

S, P, H, D, C, N = 1, 2, 6, 5, 4, 21
#S, P, H, D, C, N = 1, 2, 6, 5, 4, 35
#S, P, H, D, C, N = 2, 3, 8, 4, 6, 40

Em seguida, declaramos a matriz de alocação, $X$, e uma variável $T$ que irá representar a carga horária semanal mínima de cada professor (com as restrições adequadas).

### Exercício 1

Complete a declaração da matriz de alocação $X$ como um dicionário.

In [85]:
X={}
for p in range(P):
    for s in range(S):
        for d in range(D):
            for h in range(H):
                X[p,s,d,h]=Int(str(p)+'_'+str(s)+'_'+str(d)+'_'+str(h))
                horario.add(X[p,s,d,h]>=0,X[p,s,d,h]<=1)
T=Int("T")
horario.add(T>=0,T<=D*C)
                
# completar

Passamos agora à modelação das restrições e à sua introdução no Z3.

A restrição

1. A carga horária diaria máxima de cada professor é $C$

pode expressar-se da seguinte forma:

$$\forall_{d< D} \cdot \forall_{p< P} \cdot \quad \sum_{h< H,\,s< S} x_{p,s,d,h} \leq C$$

In [86]:
for d in range(D):
    for p in range(P):
        horario.add(Sum([X[p,s,d,h] for h in range(H) for s in range(S)]) <= C)

### Exercício 2

Apresente as fórmulas que modelam as restantes restrições e acrescente-as ao problema `horario`.

2. Cada sala tem alocada, no máximo, um professor.

$$
\forall_d \cdot \forall_h \cdot \forall_s \cdot \quad \sum_p X_{p,s,d,h} \leq 1
$$



In [87]:
# completar
for d in range(D):
    for h in range(H):
        for s in range(S):
            horario.add(Sum([X[p,s,d,h] for p in range(P)])<=1)

3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.
$$
\forall_d \cdot \forall_h \cdot \forall_p \cdot \quad \sum_s X_{p,s,d,h} \leq 1
$$

In [88]:
# completar
for d in range(D):
    for h in range(H):
        for p in range(P):
            horario.add(Sum([X[p,s,d,h] for s in range(S)])<=1)

4. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).
$$
\forall_p \cdot \quad T \leq \sum_{s,d,h} X_{p,s,d,h} \leq T+2
$$

In [89]:
# completar
for p in range(P):
    horario.add(Sum([X[p,s,d,h] for d in range(D) for h in range(H) for s in range(S)])>=T)
    horario.add(Sum([X[p,s,d,h] for d in range(D) for h in range(H) for s in range(S)])<=T+2)

5. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.
$$
\forall_p \cdot \forall_d \cdot \forall_{h<H-1} \cdot \forall_s \cdot \forall_{s' \neq s} \cdot X_{p,s,d,h} + X_{p,s',d,h+1} \leq 1
$$


In [90]:
# completar
for p in range(P):
    for d in range(D):
        for h in range(H-1):
            for s in range(S):
                for s_ in range(S):
                    if s!=s_:
                        horario.add(X[p,s,d,h]+X[p,s_,d,h+1]<=1)

6. O número mínimo de aulas por semana é $N$.
$$
\sum_{p,s,d,h} X_{p,s,d,h} \geq N
$$


In [91]:
# completar
horario.add(Sum(list(X.values()))>=N)

### Exercício 3

Finalize a resolução do problema procurando uma solução, e imprimindo o número de aulas por semana, caso exista.

In [96]:
# completar
r=horario.check()
print(r)
if r== sat:
    m=horario.model()
    print(m.eval(Sum(list(X.values()))))

sat
21


### Exercício 4

Defina funções para construir os horarios de professores e salas individuais, e para apresentar de forma legível esses horários.

In [97]:
# completar
for p in range(P):
    print("Professor",p)
    for d in range(D):
        for h in range(H):
            for s in range(S):
                if m[X[p,s,d,h]]==1:
                    print("Dia",d,"Hora",h,"Sala",s)
for s in range(S):
    print("Sala",s)
    for d in range(D):
        for h in range(H):
            for p in range(P):
                if m[X[p,s,d,h]]==1:
                    print("Dia",d,"Hora",h,"Professor",p)                    


Professor 0
Dia 1 Hora 2 Sala 0
Dia 1 Hora 3 Sala 0
Dia 1 Hora 4 Sala 0
Dia 1 Hora 5 Sala 0
Dia 2 Hora 1 Sala 0
Dia 2 Hora 2 Sala 0
Dia 3 Hora 4 Sala 0
Dia 3 Hora 5 Sala 0
Dia 4 Hora 0 Sala 0
Dia 4 Hora 1 Sala 0
Professor 1
Dia 0 Hora 5 Sala 0
Dia 1 Hora 0 Sala 0
Dia 1 Hora 1 Sala 0
Dia 2 Hora 4 Sala 0
Dia 3 Hora 0 Sala 0
Dia 3 Hora 1 Sala 0
Dia 3 Hora 2 Sala 0
Dia 3 Hora 3 Sala 0
Dia 4 Hora 2 Sala 0
Dia 4 Hora 4 Sala 0
Dia 4 Hora 5 Sala 0
Sala 0
Dia 0 Hora 5 Professor 1
Dia 1 Hora 0 Professor 1
Dia 1 Hora 1 Professor 1
Dia 1 Hora 2 Professor 0
Dia 1 Hora 3 Professor 0
Dia 1 Hora 4 Professor 0
Dia 1 Hora 5 Professor 0
Dia 2 Hora 1 Professor 0
Dia 2 Hora 2 Professor 0
Dia 2 Hora 4 Professor 1
Dia 3 Hora 0 Professor 1
Dia 3 Hora 1 Professor 1
Dia 3 Hora 2 Professor 1
Dia 3 Hora 3 Professor 1
Dia 3 Hora 4 Professor 0
Dia 3 Hora 5 Professor 0
Dia 4 Hora 0 Professor 0
Dia 4 Hora 1 Professor 0
Dia 4 Hora 2 Professor 1
Dia 4 Hora 4 Professor 1
Dia 4 Hora 5 Professor 1


### Exercício 5

Explore o comportamento do modelo pela variação dos parâmetros $S$, $P$, $H$, $D$, $C$ e $N$.

### Exercício 6

Queremos agora acrescentar a seguinte regra no funcionamento do centro de estudos:

> Cada professor tem de ter um dia da semana em que não dá aulas.

Esta *obrigação* poderia ser expressa matematicamente, de forma direta, por
$$
\forall_{p<P}.\exists_{d<D}. \quad \sum_{s<S,h<H} x_{p,s,d,h} = 0
$$
ou, em alternativa, pela seguinte expressão
$$
\forall_{p<P}. \quad \bigvee_{d<D} \big(\sum_{s<S,h<H} x_{p,s,d,h} = 0\big) 
$$

Contudo a disjunção não tem uma representação direta nos solvers para programação inteira. Para a implementar podemos acrescentar uma família de variáveis binárias $y_{p,d}$ que indicam se o professor $p$ dá aulas no dia $d$, com a seguinte restrição que limita o número máximo de dias em que o professor dá aulas.

$$\forall_{p<P} \cdot \quad \sum_{d<D} y_{p,d} < D $$

O valor de  $y_{p,d}$ deve também de alguma forma limitar superiormente as aulas que o professor $p$ dá no dia $d$.
Apresente uma fórmula que modele esta nova limitação.
$$
\forall_p \cdot \forall_h \cdot \quad \sum_{s,h} X_{p,s,d,h} \leq y_{p,d} X C
$$

In [92]:
# completar
Y={}
for p in range(P):
    for h in range(H):
        Y[p,h]=Int(str((p,h)))
        horario.add(Y[p,h] >=0 , Y[p,h]<=1)
        horario.add(Sum([X[p,s,d,h] for h in range(H) for s in range(S)])<=C*Y[p,h])

Acrescente agora estas fórmulas ao problema `horario`, e encontre nova solução.

In [ ]:
# completar